# 미니 프로젝트: 장애인 콜택시 대기시간 예측 모델 개발
> 일기 예보에 따라 장애인 콜택시 대기 시간을 예측하는 모델을 개발하여 장애인 이동권 개선

<img src = "https://www.sisul.or.kr/open_content/calltaxi/images/visual_img.jpg"/>

In [1]:
# pandas == 1.5.3
# numpy == 1.22.4
# matplotlib == 3.7.1
# seaborn == 0.12.2
# xgboost == 1.7.5

## 단계 1. 데이터 전처리

In [2]:
# 라이브러리 로딩
import pandas as pd
import numpy as np
import joblib

### (1) 데이터셋 불러오기
- 장애인 콜택시 운행 정보 : open_data.csv
- 날씨 데이터 : weather.csv

In [3]:
taxi = pd.read_csv('Dataset/open_data.csv')
weather = pd.read_csv('Dataset/weather.csv')

### (2) Meta 정보 분석

In [4]:
# 데이터 확인
display(taxi.head())
print()
print('-' * 30)
print()
display(weather.head())

,기준일,차량운행,접수건,탑승건,평균대기시간,평균요금,평균승차거리
0,2015-01-01,213,1023,924,23.2,2427,10764
1,2015-01-02,420,3158,2839,17.2,2216,8611
2,2015-01-03,209,1648,1514,26.2,2377,10198
3,2015-01-04,196,1646,1526,24.5,2431,10955
4,2015-01-05,421,4250,3730,26.2,2214,8663



------------------------------



,Date,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2)
0,2012-01-01,0.4,-6.6,0.0,77.0,45.0,4.90
1,2012-01-02,-1.2,-8.3,0.0,80.0,48.0,6.16
2,2012-01-03,-0.4,-6.6,0.4,86.0,45.0,4.46
3,2012-01-04,-4.6,-9.5,0.0,66.0,38.0,8.05
4,2012-01-05,-1.4,-9.6,0.0,71.0,28.0,9.14


In [5]:
# 데이터 Sample, Feature 개수 확인
print('Taxi Dataset:', taxi.shape)
print('Weather Dataset:', weather.shape)

Taxi Dataset: (2922, 7)
Weather Dataset: (4018, 7)


In [6]:
# Meta 정보 확인
taxi.info()
print()
print('-' * 30)
print()
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2922 entries, 0 to 2921
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   기준일     2922 non-null   object 
 1   차량운행    2922 non-null   int64  
 2   접수건     2922 non-null   int64  
 3   탑승건     2922 non-null   int64  
 4   평균대기시간  2922 non-null   float64
 5   평균요금    2922 non-null   int64  
 6   평균승차거리  2922 non-null   int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 159.9+ KB

------------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4018 entries, 0 to 4017
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             4018 non-null   object 
 1   temp_max         4018 non-null   float64
 2   temp_min         4018 non-null   float64
 3   rain(mm)         4018 non-null   float64
 4   humidity_max(%)  4018 non-null   float64
 5   humidity_min(%)  4018 non-null 

### (3) 데이터셋 병합
- (택시 데이터셋 + 날씨 데이터셋) 병합

In [7]:
# 두 데이터셋을 Merge 하기 위해 변수명을 영어로 통일
taxi.rename(columns = {'기준일' : 'Date',
                       '차량운행': 'vehicle_cnt',
                       '접수건': 'reg_cnt',
                       '탑승건' : 'ride_cnt',
                       '평균대기시간' : 'average_wait_time',
                       '평균요금' : 'average_fare',
                       '평균승차거리' : 'average_dist'}, inplace = True)

In [8]:
df = pd.merge(taxi, weather, on='Date')

### (4) New Feature 생성
- 새로운 시각에서 데이터를 분석하기위해 Feature 생성

#### 날짜 관련 Feature
- 월, 년, 요일, 계절과 대시시간 분석을 위해 새로운 Feature 추가

In [9]:
# 날짜 Type 변수 형변환
df['Date'] = pd.to_datetime(df['Date'])

In [10]:
# 월
df['month'] = df['Date'].dt.month

# 년
df['year'] = df['Date'].dt.year

# 요일
df['weekday'] = df['Date'].dt.day_name()
df['weekday'] = pd.Categorical(df['weekday'], categories=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])

# 계절
df['season'] = np.where(df['month'].isin([3,4,5]), 'Spring',
                        np.where(df['month'].isin([6,7,8]), 'Summer',
                                    np.where(df['month'].isin([9,10,11]), 'Fall', 'Winter')))
df['season'] = pd.Categorical(df['season'], categories=['Spring','Summer','Fall','Winter'])

#### 주말 Feature

In [11]:
df['week'] = np.where(df['weekday'].isin(['Saturday','Sunday']), 1, 0)

#### 공휴일 Feature
- 공휴일이 콜택시 대기시간에 영향을 미칠 수 있다 생각하여 코로나 데이터셋을 추가하여 새로운 Feature 추가

In [12]:
# !pip install workalendar

In [13]:
from workalendar.asia import SouthKorea

# 공휴일 정보 가져오기
cal = SouthKorea()
holiday_df = pd.DataFrame()

# 공휴일 DataFrame 생성
for year in range(2015, 2023):
    
    holiday_list = cal.holidays(year)
    holiday_df = pd.concat([holiday_df, pd.DataFrame(holiday_list)], axis=0)
    

holiday_df.columns = ['Date', 'holiday']
holiday_df['Date'] = pd.to_datetime(holiday_df['Date'])
holiday_df['holiday'] = "Yes"

In [14]:
df = pd.merge(df, holiday_df, how = 'left')
df['holiday'] = df['holiday'].fillna("No")

#### Covid Feature
- 코로나가 콜택시 대기시간에 영향을 미칠 수 있다 생각하여 코로나 데이터셋 추가하여 새로운 Feature 추가

In [15]:
covid = pd.read_csv('Dataset/covid19.csv')
covid.columns = ['Date', 'covid_sum']
covid['Date'] = pd.to_datetime(covid['Date'])

df = pd.merge(df, covid, how = 'left')
df['covid_sum'] = df['covid_sum'].fillna(0)

#### 주 평균 대기시간 Feature

In [16]:
# 주 평균 대기시간 (rolling -> 7)
df['week_average_wait_time'] = df[['average_wait_time']].rolling(window=7).mean()
df['week_average_wait_time'] = df['week_average_wait_time'].fillna(df['week_average_wait_time'].mean())  # 누락 값 평균 값

#### 탑승률 Feature
- 탑승률과 대기시간이 상관관계가 있을것이라 판단하여 Feature 추가

In [17]:
# 차량 접수자 중 실제 탑승자
df['ride_ratio'] = df['reg_cnt'] / df['ride_cnt'] * 100

#### 강수 여부
- 강수량이 아닌 강수 여부를 확인하는 Feature 추가

In [18]:
# 비가 온날과 오지 않은 날 구분
df['rain'] = np.where(df['rain(mm)'] > 0, 1, 0)

### (6) NaN 값 제거

In [19]:
df.isna().sum()

Date                      0
vehicle_cnt               0
reg_cnt                   0
ride_cnt                  0
average_wait_time         0
average_fare              0
average_dist              0
temp_max                  0
temp_min                  0
rain(mm)                  0
humidity_max(%)           0
humidity_min(%)           0
sunshine(MJ/m2)           0
month                     0
year                      0
weekday                   0
season                    0
week                      0
holiday                   0
covid_sum                 0
week_average_wait_time    0
ride_ratio                0
rain                      0
dtype: int64

### (7) 가변수화
- 모델 학습을 위해 숫자로 변경

In [20]:
# # 가변수화
# df = pd.get_dummies(df, columns = ['weekday', 'season', 'holiday'], drop_first = True, dtype = int)
# df.info()

### (8) 전처리 데이터 저장


In [21]:
# 저장
joblib.dump(df, 'Dataset/preprocess_data.pkl')

['Dataset/preprocess_data.pkl']